# PTT爬蟲

In [53]:
!mkdir -p /tmp/ptt_data/list_page

In [54]:
!mkdir -p /tmp/ptt_data/detail_page

In [55]:
import requests

def get_total_page():
    resp = requests.get("https://www.ptt.cc/bbs/Gossiping/index.html", headers={'cookie': "over18=1"})
    text = resp.text
    return int(text.split('a class="btn wide" href="/bbs/Gossiping/index')[2].split('.html">&lsaquo; 上頁</a>')[0]) + 1

def get_page(url):
    """
    args: 
          url<str>: url to crawl
    return:
          res<str>: text of the webpage
    """
    resp = requests.get(url, headers={'cookie': "over18=1"})
    return resp.text

def save2file(filename, text):
    """
    args:
          filename<str>: filename of the file
          text<str>: text content to be saved
    return:
          
    """
    with open(filename, 'w') as f:
        f.write(text)

In [56]:
url_template = "https://www.ptt.cc/bbs/Gossiping/index%s.html"
total_page = get_total_page()
pages_to_crawl = 2
for page in range(total_page, total_page - pages_to_crawl,-1):
    url = url_template%page
    print("[INFO]crawling %s"%url)
    text = get_page(url)
    save2file('/tmp/ptt_data/list_page/' + "%s.html"%page, text)

[INFO]crawling https://www.ptt.cc/bbs/Gossiping/index39057.html
[INFO]crawling https://www.ptt.cc/bbs/Gossiping/index39056.html


In [57]:
# glob is like ls for Python
# finds file paths
import glob

glob.glob('/tmp/ptt_data/list_page/*')

res = []
for filename in glob.glob('/tmp/ptt_data/list_page/*'):
    with open(filename) as f:
        html = f.read()
        detail_urls = re.findall('''/bbs/Gossiping/M.+html''', html)
        res.extend(detail_urls)

res

['/bbs/Gossiping/M.1522112372.A.E71.html',
 '/bbs/Gossiping/M.1522112378.A.AE4.html',
 '/bbs/Gossiping/M.1522112405.A.B2C.html',
 '/bbs/Gossiping/M.1522112478.A.5CD.html',
 '/bbs/Gossiping/M.1522112485.A.921.html',
 '/bbs/Gossiping/M.1522112503.A.1BD.html',
 '/bbs/Gossiping/M.1522112553.A.D9A.html',
 '/bbs/Gossiping/M.1522112604.A.138.html',
 '/bbs/Gossiping/M.1522112646.A.450.html',
 '/bbs/Gossiping/M.1522112685.A.A97.html',
 '/bbs/Gossiping/M.1522112686.A.54F.html',
 '/bbs/Gossiping/M.1522112726.A.8E3.html',
 '/bbs/Gossiping/M.1522112834.A.FA2.html',
 '/bbs/Gossiping/M.1522112856.A.FB2.html',
 '/bbs/Gossiping/M.1522112875.A.207.html',
 '/bbs/Gossiping/M.1522112889.A.B16.html',
 '/bbs/Gossiping/M.1522112890.A.BFE.html',
 '/bbs/Gossiping/M.1522112913.A.FD0.html',
 '/bbs/Gossiping/M.1522112922.A.949.html',
 '/bbs/Gossiping/M.1522108695.A.B98.html',
 '/bbs/Gossiping/M.1522108824.A.79D.html',
 '/bbs/Gossiping/M.1522108900.A.E6B.html',
 '/bbs/Gossiping/M.1522108938.A.21D.html',
 '/bbs/Goss

In [59]:
for link in res:
    filename = link.split('/')[-1].replace('.html','').replace('.', '_') + '.html'
    url = 'https://www.ptt.cc' + link
    print(url)
    
    # using the fucntions defined above ^^^^^^^^^^^^^
    text = get_page(url)
    save2file('/tmp/ptt_data/detail_page/' + filename, text)

In [7]:
# GET Request
import requests

url = "http://ianchenhq.com"

resp = requests.get(url)

resp

<Response [200]>

In [8]:
resp.status_code

200

In [214]:
resp.text

# HTML Parsing

In [213]:
import glob

detail_page = glob.glob('/tmp/ptt_data/detail_page/*')[0]

print(detail_page)

with open(detail_page) as f:
    html = f.read()
    
html

/tmp/ptt_data/detail_page/M_1522114145_A_ADB.html


In [10]:
!pip install beautifulsoup4

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [20]:
from bs4 import BeautifulSoup


soup = BeautifulSoup(html, 'lxml') # html5lib

In [22]:
soup.select_one('span.article-meta-value')

<span class="article-meta-value">oridio (新手上路)</span>

In [30]:
metas = soup.select('span.article-meta-value')

metas # [<Tag>, <Tag>, <Tag>, <Tag>]

[<span class="article-meta-value">oridio (新手上路)</span>,
 <span class="article-meta-value">Gossiping</span>,
 <span class="article-meta-value">[新聞] 綠執政親日 農委會成重災區</span>,
 <span class="article-meta-value">Tue Mar 27 09:29:02 2018</span>]

In [31]:
"""
{
"title": "[新聞] 綠執政親日 農委會成重災區",
"author": "oridio (新手上路)",
"board": "Gossiping"
}

"""
article_dict = {}

# metas[0] -> <Tag>
# metas[0].text => Tag.text -> <str>
article_dict['author'] = metas[0].text
article_dict['board']  = metas[1].text
article_dict['title']  = metas[2].text
article_dict['dt']     = metas[3].text

article_dict

{'author': 'oridio (新手上路)',
 'board': 'Gossiping',
 'dt': 'Tue Mar 27 09:29:02 2018',
 'title': '[新聞] 綠執政親日 農委會成重災區'}

In [35]:
import re

re.findall(', 來自: 39.9.68.12\n', html)

[', 來自: 39.9.68.12\n']

In [36]:
re.findall(', 來自: (39.9.68.12)\n', html)

['39.9.68.12']

In [37]:
re.findall(', 來自: (\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3})\n', html)

['39.9.68.12']

In [39]:
article_dict['ip'] = re.findall(', 來自: (\d{1,3}.\d{1,3}.\d{1,3}.\d{1,3})\n', html)[0]
article_dict

{'author': 'oridio (新手上路)',
 'board': 'Gossiping',
 'dt': 'Tue Mar 27 09:29:02 2018',
 'ip': '39.9.68.12',
 'title': '[新聞] 綠執政親日 農委會成重災區'}

In [47]:
main_content = soup.select_one('#main-content')

for tag in main_content.select('.article-metaline'):
    tag.extract()
    
for tag in main_content.select('.article-metaline-right'):
    tag.extract()

In [212]:
main_content

In [49]:
for tag in main_content.select('div.push'):
    tag.extract()

In [211]:
main_content

In [210]:
article_dict['content'] = main_content.text.strip()

article_dict

# 591

爬蟲神器

我寫的，歡迎回報BUG或是各種建議

https://chrome.google.com/webstore/detail/quick-http-inspector/holkomabobpkbfdfnjomglmcgaeeojpg?hl=en

In [140]:
# 有BUG，沒辦法換Region

import requests

url = "https://rent.591.com.tw/home/search/rsList"

qs = {"is_new_list":1,
      "type":1,
      "kind":0,
      "searchtype":1,
      "firstRow": 0, # pagination, increment this to set the OFFSET
      "region":15}

res = requests.get(url,
                   params=qs,
                   headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.162 Safari/537.36'})

res.json()

{'data': {'biddings': [],
  'data': [{'addInfo': '',
    'addition2': 0,
    'addition3': 0,
    'addition4': 1,
    'addr_number_name': '',
    'address': '新生北路二段72巷環..',
    'address_img': '環境優美近捷運壹樓住家出租',
    'address_img_title': '環境優美近捷運壹樓住家出租',
    'alley_name': '72巷',
    'allfloor': 11,
    'area': 25,
    'balcony_area': None,
    'browsenum': 267,
    'browsenum_all': 692,
    'browsenum_name': '昨日瀏覽',
    'cartplace': '',
    'cases_id': 0,
    'cases_name': '',
    'checkstatus': 1,
    'closed': 0,
    'comment_class': '',
    'comment_ltime': 1521969212,
    'comment_total': 1,
    'comment_unread': 1,
    'condition': 'icebox,cold,hotwater,naturalgas,landpost,cook,balcony_0,wardrobe',
    'cover': 'https://hp1.591.com.tw/house/active/2018/03/18/152136253900017206_210x158.crop.jpg',
    'distance': 0,
    'filename': 'https://hp1.591.com.tw/house/active/2018/03/18/152136253900017206_210x158.crop.jpg',
    'floor': 1,
    'floor2': 0,
    'floorInfo': '樓層：1/11',
    'fullad

In [169]:
import requests
import json

url = "https://rent.591.com.tw/home/search/rsList"
#data = 
headers = json.loads(r'''{
    "Cookie": "urlJumpIp=6;",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"
}''')

resp = requests.get(url, headers=headers)

In [209]:
resp.json()

# PCC Web 政府採購網

## Session

In [198]:
import requests
import json

url = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance"
data = json.loads(r'''{
    "method": "search",
    "searchMethod": "true",
    "searchTarget": "ATM",
    "orgName": "",
    "orgId": "",
    "hid_1": "1",
    "tenderName": "",
    "tenderId": "",
    "tenderStatus": "4,5,21,29",
    "tenderWay": "",
    "awardAnnounceStartDate": "107/03/26",
    "awardAnnounceEndDate": "107/03/26",
    "proctrgCate": "",
    "tenderRange": "",
    "minBudget": "",
    "maxBudget": "",
    "item": "",
    "hid_2": "1",
    "gottenVendorName": "",
    "gottenVendorId": "",
    "hid_3": "1",
    "submitVendorName": "",
    "submitVendorId": "",
    "location": "",
    "execLocationArea": "",
    "priorityCate": "",
    "isReConstruct": "",
    "btnQuery": "查詢"
}''')
headers = json.loads(r'''{
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7",
    "Cache-Control": "max-age=0",
    "Connection": "keep-alive",
    "Content-Length": "429",
    "Content-Type": "application/x-www-form-urlencoded",
    "Host": "web.pcc.gov.tw",
    "Origin": "http://web.pcc.gov.tw",
    "Referer": "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"
}''')

resp = requests.post(url, data=data, headers=headers)
    

In [207]:
resp.text

In [203]:
init_cookies = resp.cookies.get_dict()
init_cookies

{'JSESSIONID': '0000MVlovZg5iB4QsJu6-hus6Ow:14adnh85p',
 'NSC_xfc_qfstjtufodf': 'ffffffff09081f7745525d5f4f58455e445a4a423660',
 'ccguid': '201803271430336601000a1388efbd',
 'ccsession': '201803271430336601000a1388efbd',
 'cookiesession1': '5A58395AZI5O4G8MGMSIQXEXQGLSB6AE'}

In [205]:
import requests
import json

url = "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance&searchTarget=ATM&method=search&isSpdt=&pageIndex=2&execLocationArea="
headers = json.loads(r'''{
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate",
    "Accept-Language": "en-US,en;q=0.9,zh-TW;q=0.8,zh;q=0.7",
    "Connection": "keep-alive",
    "Host": "web.pcc.gov.tw",
    "Referer": "http://web.pcc.gov.tw/tps/pss/tender.do?searchMode=common&searchType=advance",
    "Upgrade-Insecure-Requests": "1",
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.181 Safari/537.36"
}''')

# reuse cookies from first request
resp = requests.get(url, headers=headers, cookies=init_cookies)
    

In [208]:
resp.text

# Datetime

In [242]:
# Parse string into Python's datetime object
# method 1: use dateutil to auto parse strings into datetime objects
# method 2: use Python's builtin datetime module to parse

In [217]:
!pip install dateutils

You are using pip version 9.0.1, however version 9.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [251]:
# method 1:

In [252]:
import dateutil.parser

In [253]:
dateutil.parser.parse('2018-03-27 15:52')

datetime.datetime(2018, 3, 27, 15, 52)

In [254]:
dt = dateutil.parser.parse("Tue Mar 27 09:29:02 2018")
dt

datetime.datetime(2018, 3, 27, 9, 29, 2)

In [255]:
# method 2:

In [256]:
from datetime import datetime

dt_str = "2018年3月27日 16時11分"
datetime.strptime(dt_str, '%Y年%m月%d日 %H時%M分')

datetime.datetime(2018, 3, 27, 16, 11)

In [257]:
# fiddling with datetime object 

In [258]:
dt.hour

9

In [259]:
dt.minute

29

In [260]:
dt.isoformat()

'2018-03-27T09:29:02'

In [261]:
# add timezone info to datetime object

In [262]:
from pytz import timezone
TPE = timezone("Asia/Taipei")
TPE # timezone object for Taipei

<DstTzInfo 'Asia/Taipei' LMT+8:06:00 STD>

In [263]:
# <timezone>.localize(<naive_datetime_obj>)
aware_dt = TPE.localize(dt)

In [264]:
aware_dt.isoformat()

'2018-03-27T09:29:02+08:00'

In [265]:
aware_dt

datetime.datetime(2018, 3, 27, 9, 29, 2, tzinfo=<DstTzInfo 'Asia/Taipei' CST+8:00:00 STD>)

In [266]:
# Convert aware datetime object into another timezone..
# <aware datetime obj>.astimezone(<target timezone>)
aware_dt.astimezone(timezone("US/Eastern"))
aware_dt.astimezone(timezone("UTC"))

datetime.datetime(2018, 3, 27, 1, 29, 2, tzinfo=<UTC>)

In [267]:
# Convert "aware datetime obj" to unix timestamp
aware_dt.timestamp()

1522114142.0

In [269]:
aware_dt.isoformat()

'2018-03-27T09:29:02+08:00'